In [1]:

from kafka import KafkaProducer, KafkaConsumer
from time import sleep
import json
from datetime import datetime
from confluent_kafka import Producer
import socket
import pymongo
from pymongo.errors import BulkWriteError
from datetime import datetime

from pymongo import MongoClient, ASCENDING, DESCENDING

from pprint import pprint
import time

from time import sleep
from bson.objectid import ObjectId 
import subprocess
from itertools import combinations, combinations_with_replacement
from EtudeBinance import Makethefile,getpairs

{'id': 790420939, 'price': '3147.53000000', 'qty': '0.01490000', 'quoteQty': '46.89819700', 'time': 1648390887130, 'isBuyerMaker': False, 'isBestMatch': True}


In [2]:
symboles=list(filter(('').__ne__, getpairs()))
listesymboles=["BTC","ETH","USDT","USD","BNB","XRP","ADA","DOGE","LTC","BCH","SOL","ETC","LUNA","AVAX","SHIB","DOT","MATIC","AAVE","UNI","VRA","USDC","ADX","USDC","APE","JASMY","GALA","WAVES","LRC"]
temp = combinations(listesymboles, 2)
newlist=[]
for j in list(temp):
    newlist.append(j[0]+j[1])
    newlist.append(j[1]+j[0])
for i in newlist:
    if i not in symboles:
        newlist.remove(i)
print("len",len(newlist))
print("len",len(symboles))

len 388
len 1985


# Définition des paramètres

**Producer** permet d'envoyer des données tandis que **consumer** permet de lire en direct.

Le client de mongo permet de stocker ces valeurs dans notre database.

In [3]:

conf = {'bootstrap.servers': "host1:9092,host2:9092",
        'client.id': socket.gethostname()}

producerconnect = Producer({'bootstrap.servers': 'localhost:9092'})
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         api_version=(0,10,2))
consumer = KafkaConsumer(
    'ApiBinance',
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
     auto_commit_interval_ms=300,
     enable_auto_commit=True,
     group_id='my-group')

client = MongoClient("localhost", 27017)
db = client.Binance
collection = db.symbol
db.symbol.delete_many({})

# Insertion

On a récupérer les données du fichier. On utilise Kafka Connect pour faire le lien entre mongodb et les topics.

On insere les données dans un topics.

In [ ]:
# from datetime import datetime

# # datetime object containing current date and time
# now = datetime.now()
 
# print("now =", now)

# # dd/mm/YY H:M:S
# dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
# print("date and time =", dt_string)

In [ ]:
def ApacheRemplissage():
	listeInsertion=[]

	with open("fichier.txt", "r") as fichier:
		
		lines = fichier.readlines()
		for line in lines:
			listeInsertion.append(line)

	list1=[]
	list1=listeInsertion[0].split(',WEEE')

	del list1[-1]

	jsone=[]

	for i in list1:
		try:
			jsone.append(json.loads(i))
		except ValueError:
			print("error for",i,ValueError)

	for i in jsone:
		producer.send('ApiBinance',bytes(str(i), encoding='utf-8'))
		sleep(0.5)

On a inséré les données dans kafka topics. Maintenant, via Kafka connect on récupére les données puis on les met dans mongodb

**Kafka Connect**

Lire des données & on met les données dans MongoDB.

In [ ]:
from confluent_kafka import Consumer, KafkaError

client='client-'
compteur=0
def conso():
    settings = {
        'bootstrap.servers': 'localhost:9092',
        'group.id': 'mygroup2',
        'client.id': 'client-'+str(compteur),
        'enable.auto.commit': True,
        'session.timeout.ms': 6000,
        'default.topic.config': {'auto.offset.reset': 'smallest'}
    }

    c = Consumer(settings)

    c.subscribe(['ApiBinance'])

    try:
        while True:
            msg = c.poll(0.1)
            if msg is None:
                continue
            elif not msg.error():
                print('Received message: {0}'.format(msg.value()))
                inputdonnee=collection.insert_one(json.loads(msg.value()))
                print("New input in our collection")
            elif msg.error().code() == KafkaError._PARTITION_EOF:
                print('End of partition reached {0}/{1}'
                    .format(msg.topic(), msg.partition()))
            else:
                print('Error occured: {0}'.format(msg.error().str()))

    except KeyboardInterrupt:
        pass

    finally:
        c.close()
    compteur+=1

On actualise le topic continuellement, pour supprimer les anciennes donées grâce aux données présentes dans mongoDB.

In [ ]:
def acked(err, msg):
    if err is not None:
        print("Failed to deliver message: {0}: {1}"
              .format(msg.value(), err.str()))
    else:
        print("Message produced: {0}".format(msg.value()))

def ProducerTerminal():
    p = Producer({'bootstrap.servers': 'localhost:9092'})
    cursor = collection.find()
    try:
        for record in cursor:
            p.produce('ApiBinance', record, callback=acked)
            p.poll(0.5)

    except KeyboardInterrupt:
        pass

    p.flush(30)
    db.symbol.delete_many({})
    conso()

On fait des requetes

In [ ]:
def requetesMongoDB():
    return 5

In [ ]:
print("Bonjour, bienvenue dans le menu du Projet Apache-MongoDB : ")
index=True
while index:
    print("\n")
    print("-------------start-------------")
    print("\n")
    case=input("Do you want to read the data in the Apache topic and MongoDB (in this case, the database will be filled automatically)(Press 1) or execute some request to MongoDB (Press 2) or  exit (Press 3) > ")
    print("\n")
    if case == "1":
        start = time.time()
        ApacheRemplissage()
        conso()
        elapsed = end - start
        end = time.time()
        print(f'Temps d\'exécution pour le remplissage : {elapsed}s')
    elif case =="2":
        requetesMongoDB()
    elif case =="3":
        index=False
        print("Au revoir !")
    else:
        print("Something, you don't press an avalaible numbers, please restart")
       
    print("-------------end---------------")
